In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# Realiza alguns imports para a especificação de tipos
from typing import Self

In [13]:
class Dataset:
    def __init__( self, data: pd.DataFrame, *, label_column : int = -1, column_names : list = None ) -> None:
        """
        Inicializa a classe Dataset.

        Args:
            data (DataFrame): DataFrame com os dados do conjunto de dados;
            label_column (int): Inteiro que especifica a posição da coluna de rótulo;
            column_names (list): Lista com as strings de cada coluna.
        """

        self.data = data                    # Guarda o DataFrame do conjunto de dados
        self.label_column = label_column    # Armazena o índice para a coluna de rótulo
        self.column_names = column_names    # Armazena a lista com os rótulos das colunas do DataFrame

        # Se for especificado, define o nome das colunas
        if self.column_names is not None:
            self._set_column_names()

    def _set_column_names( self ) -> None:
        """
        Atualiza os rótulos das colunas do DataFrame, conforme o valor no atributo column_names.
        """

        self.data.columns = self.column_names

    def shuffle( self ) -> Self:
        """
        Retorna uma nova instância da classe com os dados embaralhados.
        """

        # Obtém um DataFrame embaralhado e com os índices resetados
        data = self.data.sample(frac=1).reset_index( drop=True )

        # Retorna uma nova instância da classe
        return self.__class__( data, label_column = self.label_column, column_names = self.column_names )
    
    @classmethod
    def from_file( cls, filepath : str, comment_marker : str = "#", missing_marker : str = "?", *, label_column : int = -1, column_names : list = None ) -> None:
        """
        Inicializa a classe especificando o caminho de um arquivo.

        Args:
            filepath (str): Caminho para o arquivo CSV;
            comment_marker (str): Marcador para linhas de comentário (serão ignoradas);
            missing_marker (str): Marcador para elementos faltantes (o pandas irá substituir valores por NaN);
            label_column (int): Inteiro que especifica a posição da coluna de rótulo;
            column_names (list): Lista com as strings de cada coluna.
        """

        # Lê o arquivo CSV (Não existe linhas de header no arquivo)
        data = pd.read_csv( filepath, header=None, comment=comment_marker, na_values=missing_marker )

        # Remover elementos faltantes
        data = data.dropna()

        # Retorna uma chamada para o construtor normal da classe
        return cls( data, label_column = label_column, column_names = column_names )
    
    @property
    def X( self ) -> pd.DataFrame:
        """
        Retorna as colunas de atributos
        """

        # Se a coluna de resultados estiver em uma posição diferente da coluna final, compõe o DataFrame com os atributos
        if self.label_column != -1:
            return pd.concat( [self.data.iloc[:, :self.label_column],  self.data.iloc[:, self.label_column+1:]], axis=1 )
        
        # Caso mais simples, a coluna de label está no final
        return self.data.iloc[:, :self.label_column]
    
    @property
    def y( self ) -> pd.Series:
        """
        Retorna a coluna de rótulo
        """

        return self.data.iloc[:, self.label_column]
    
    @property
    def shape( self ) -> tuple:
        """
        Retorna as dimensões do Dataset

        Returns:
            tuple: Retorna uma tupla na forma (m,n,k), onde m é o número de instâncias, n é o número de atributos e k é o número de classes.
        """

        m, n = self.data.shape
        k = len( set( self.y ) )

        return (m, n, k)

In [20]:
path_dataset = r"datasets\dermatology.data"    # caminho do arquivo
column_names = ["sepal length", "sepal width", "petal length", "petal width", "class"] # Nome dos campos

data = Dataset.from_file( path_dataset, label_column=-1 )
data.shape

(358, 35, 6)